In [21]:
from search import *
import random

In [22]:
import copy

import re

from search import *


class Movix(Problem):
    N = 0
    Y = 0
    muros = []

    def __init__(self, N, Y, initial, goal=None, muros = []):
        '''Inicializacion de nuestro problema.'''
        Problem.__init__(self, initial, goal)
        self.N = N
        self.Y = Y
        self._actions = []
        self.muros = muros

        for i in range(1, N):
            self._actions.append(('V', (0, i)))
            self._actions.append(('V', (0, -i)))

        for i in range(1, N):
            self._actions.append(('H', (i, 0)))
            self._actions.append(('H', (-i, 0)))

        for i in range(-1, 2):
            for j in range(-1, 2):
                if i != 0 or j != 0:
                    self._actions.append(('L', (i, j)))
                    if i == 0 or j == 0:
                        self._actions.append(('S', (2 * i, 2 * j)))

    def mostrarAcciones(self):
        print(self._actions)

    def actions(self, s):
        actions_list = []
        for a in self._actions:
            for pos in s[0][a[0]]:
                if self._is_valid(a, pos, s):
                #if self.esValido(a, pos, s):
                    actions_list.append("Mover ficha {} de ({},{}) a ({},{})".format(a[0], pos[0], pos[1], (pos[0] + a[1][0]), (pos[1] + a[1][1])))

        return actions_list

    #Nuevo is_valid con la condicion adicional de los muros
    def _is_valid(self, a, pos, s):
        fila, columna = (pos[0] + a[1][0], pos[1] + a[1][1])
        #Compruebo que la posicion este dentro de las dimensiones del tablero, que no hay ninguna ficha
        #en dicha posicion y que tampoco haya ningun muro
        if (fila < self.N and fila >= 0 and columna < self.N and columna >= 0 and 
            (fila not in s[1] or columna not in s[1][fila]) and self.muros.count((fila, columna)) == 0):
            match a[0]:
                case 'V':
                    return not(fila in s[1] and (any(pos[1] < y < columna for y in s[1][fila]) or any(columna < y < pos[1] for y in s[1][fila])))
                case 'H':
                    return not(columna in s[2] and (any(pos[0] < x < fila for x in s[2][columna]) or any(fila < x < pos[0] for x in s[2][columna])))
                case 'S':
                    #Como la ficha S no puede saltar muros compruebo adicionalmente que no haya un muro en la casilla adyacente a S
                    return not(not (pos[0] + a[1][0]/2 in s[1] and (pos[1] + a[1][1]/2) in s[1][pos[0] + a[1][0]/2])) and self.muros.count((pos[0] + a[1][0]/2, pos[1] + a[1][1]/2)) == 0
                case 'L':
                    return True
        else:
            return False
        
    #Antiguo is_valid
    def esValido(self, a, pos, s):
        fil, col = (pos[0] + a[1][0], pos[1] + a[1][1])
         # Verificamos si la ficha está dentro de los límites del tablero
        if fil >= self.N or fil < 0 or col >= self.N or col < 0:
            return False  # Si alguna ficha está fuera de los límites, el estado no es válido
        if fil in s[1] and col in s[1][fil]:
            return False
        if a[0] == 'V' and fil in s[1] and (any(pos[1] < y < col for y in s[1][fil]) or any(col < y < pos[1] for y in s[1][fil])):
            return False
        if a[0] == 'H' and col in s[2] and (any(pos[0] < x < fil for x in s[2][col]) or any(fil < x < pos[0] for x in s[2][col])):
            return False
        if a[0] == 'S' and not (pos[0] + a[1][0]/2 in s[1] and (pos[1] + a[1][1]/2) in s[1][pos[0] + a[1][0]/2]):
            return False
        return True       # Si todas las fichas están dentro de los límites, y cumplen las propiedades el estado es válido

    def result(self, s, action):
        patron = r"Mover ficha (\w) de \((\d+),(\d+)\) a \((\d+),(\d+)\)"
        # Buscar los valores
        resultado = re.findall(patron, action)

        if resultado:
            ficha, x1, y1, x2, y2 = resultado[0]
            x1, y1 = int(x1), int(y1)
            x2, y2 = int(x2), int(y2)

            new_s = copy.deepcopy(s)

            new_s[0][ficha].remove((x1, y1))
            #no deberia hacer falta comprobar con estos ifs 
            if y1 in new_s[2] and x1 in new_s[2][y1]:
                new_s[2][y1].pop(x1)
            if x1 in new_s[1] and y1 in new_s[1][x1]:
                new_s[1][x1].pop(y1)

            
            if x1 in new_s[1] and len(new_s[1][x1]) == 0:
                del new_s[1][x1]
            if y1 in new_s[2] and len(new_s[2][y1]) == 0:
                del new_s[2][y1]

            
            if x2 not in new_s[1]:
                new_s[1][x2] = {}

            if y2 not in new_s[2]:
                new_s[2][y2] = {}

            new_s[0][ficha].append((x2, y2))
            new_s[1][x2][y2] = ficha
            new_s[2][y2][x2] = ficha

            #nos aseguramos q todo este ordenado
            
            #return new_s
            return (new_s[0], 
                    {k: new_s[1][k] for k in sorted(new_s[1].keys())},
                    {k: new_s[2][k] for k in sorted(new_s[2].keys())})
        else:
            return None

    def goal_test(self, s):
        #linea = (len(s[1]) == 1 and max(s[2].keys()) - min(s[2].keys()) == self.Y*4 - 1) or (len(s[2]) == 1 and max(s[1].keys()) - min(s[1].keys()) == self.Y*4 - 1)
        linea = len(s[1]) == 1 or len(s[2]) == 1

        diagonal = True
        #if len(s[1]) > 0 and len(s[1][min(s[1])]) > 0:
        clave_minima = min(s[1])
        diferencia = clave_minima - min(s[1][clave_minima])
        for i in s[1]:
            if(len(s[1][i]) > 1):
                diagonal = False
                break
            elif(i - min(s[1][i]) != diferencia):
                diagonal = False
                break

        return linea or diagonal


In [23]:
N = int(input("Introduzca el numero de casillas que tendra el tablero: "))
Y = int(input("Introduzca el numero de fichas que habra en el tablero: "))
M = int(input("Cuantos muros desea que haya en el tablero: "))

while(Y > N):
    print("El numero de fichas no puede ser mayor a las dimensiones del tablero")
    N = int(input("Introduzca el numero de casillas que tendra el tablero: "))
    Y = int(input("Introduzca el numero de fichas que habra en el tablero"))

#Lista de posiciones para asegurar que no se repita ninguna posicion
posiciones = []

#Lista que almacena las posiciones de los muros
muros = []

#Diccionario que relaciona el tipo de ficha con la lista de las distintas posiciones donde estan
fichasPos = {'S' : [], 'H' : [], 'V' : [], 'L' : []}

for ficha in fichasPos:
    for i in range (0, Y):
        pos = (random.randrange(0, N), random.randrange(0, N))
        while(posiciones.count(pos) > 0):
            pos = (random.randrange(0, N), random.randrange(0, N))

        fichasPos[ficha].append(pos)

for m in range(0, M):
    muro = (random.randrange(0, N), random.randrange(0, N))
    while(posiciones.count(muro) > 0 or muros.count(muro) > 0):
        muro = (random.randrange(0, N), random.randrange(0, N))

    muros.append(muro)

#Diccionario que guarda todas las fichas que estan en una fila con la columna en la que esta y el tipo de ficha que es
xToFicha = {}
#Diccionario que guarda todas las fichas que estan en una columna con la fila en la que esta y el tipo de ficha que es
yToFicha = {}

for ficha, posiciones in fichasPos.items():
    for pos in posiciones:
        if pos[0] not in xToFicha:  xToFicha[pos[0]] = {pos[1] : ficha}
        else: xToFicha[pos[0]] = xToFicha[pos[0]] | {pos[1] : ficha}

        if pos[1] not in yToFicha:  yToFicha[pos[1]] = {pos[0] : ficha}
        else: yToFicha[pos[1]] = yToFicha[pos[1]] | {pos[0] : ficha}

#Inicializo la tupla con los tres diccionarios para almacenar el estado
estado = (fichasPos, xToFicha, yToFicha)
estado


({'S': [(4, 4)], 'H': [(4, 4)], 'V': [(4, 0)], 'L': [(3, 4)]},
 {4: {4: 'H', 0: 'V'}, 3: {4: 'L'}},
 {4: {4: 'H', 3: 'L'}, 0: {4: 'V'}})

In [24]:
s = (
    {'S': [(1, 1)], 'H': [(3, 1)], 'L': [(2, 2)], 'V': [(0, 3)]},
    {0: {3: 'V'}, 1: {1: 'S'}, 2: {2: 'L'}, 3: {1: 'H'}},
    {1: {1: 'S', 3: 'H'}, 2: {2: 'L'}, 3: {0: 'V'}}
    )

#(('S', (1, 1)), ('H', (3, 1)), ('L', (2, 2)), ('V', (0, 3))) #(('_', '_', '_', '_', '_'), ('_', 'S', '_', 'H', '_'), ('_', '_', 'L', '_', '_'), ('V', '_', '_', '_', '_'), ('_', '_', '_', '_', '_')) #{'V':(0, 0), 'H':(3, 3), 'S':(2, 2), 'L':(4, 4)}

In [25]:
#Las pruebas que he hecho han sido todas con la tupla s
#Para probar con el tablero aleatorio hay que poner estado en vez de s y ya
#(No garantizo que la aleatoriedad del tablero no pueda reventar la RAM)
p = Movix(N, Y, s, muros)
#Funcion que he hecho simplemente para entender el codigo
p.mostrarAcciones()

[('V', (0, 1)), ('V', (0, -1)), ('V', (0, 2)), ('V', (0, -2)), ('V', (0, 3)), ('V', (0, -3)), ('V', (0, 4)), ('V', (0, -4)), ('H', (1, 0)), ('H', (-1, 0)), ('H', (2, 0)), ('H', (-2, 0)), ('H', (3, 0)), ('H', (-3, 0)), ('H', (4, 0)), ('H', (-4, 0)), ('L', (-1, -1)), ('L', (-1, 0)), ('S', (-2, 0)), ('L', (-1, 1)), ('L', (0, -1)), ('S', (0, -2)), ('L', (0, 1)), ('S', (0, 2)), ('L', (1, -1)), ('L', (1, 0)), ('S', (2, 0)), ('L', (1, 1))]


In [26]:
p.actions(p.initial)

['Mover ficha V de (0,3) a (0,4)',
 'Mover ficha V de (0,3) a (0,2)',
 'Mover ficha V de (0,3) a (0,1)',
 'Mover ficha V de (0,3) a (0,0)',
 'Mover ficha H de (3,1) a (4,1)',
 'Mover ficha H de (3,1) a (2,1)',
 'Mover ficha L de (2,2) a (1,2)',
 'Mover ficha L de (2,2) a (1,3)',
 'Mover ficha L de (2,2) a (2,1)',
 'Mover ficha L de (2,2) a (2,3)',
 'Mover ficha L de (2,2) a (3,2)',
 'Mover ficha L de (2,2) a (3,3)']

In [27]:
#breadth_first_tree_search(Movix(10, 1, s)).solution()

In [28]:
s = (
    {'S': [(1, 7), (3, 7)], 'H': [(1, 5), (8, 2)], 'L': [(4, 1), (3, 9)], 'V': [(4, 6), (6, 7)]},
    {1: {5: 'H', 7: 'S'}, 3: {7: 'S', 9: 'L'}, 4: {1: 'L', 6: 'V'}, 6: {7: 'V'}, 8: {2: 'H'}},
    {1: {6: 'L'}, 2: {8: 'H'}, 5: {1: 'H'}, 6: {4: 'V'}, 7: {1: 'S', 3: 'S', 6: 'V'}, 9: {3: 'L'}}
)
p = Movix(10, 2, s)

In [29]:
#breadth_first_tree_search(p).solution()